In [ ]:
import pandas
from fppylec import main, forecast, optimization
from datetime import datetime, timedelta
import plotly.express as px
import logging
import tabulate
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
%load_ext autoreload
%autoreload 2

layout = {
    'showlegend': False,
    'margin': {'b':10, 'l':20, 'r':50, 't':50},
    'font': {'size': 19},
    'xaxis': {'zerolinewidth': 2,  'zerolinecolor':'white'},
    'yaxis': {'zerolinewidth': 2, 'zerolinecolor':'white'},
    'template': 'plotly_dark',
}
px.defaults.color_discrete_sequence = px.colors.qualitative.T10

# Inputs

In [ ]:
# Filenames (correct folder)
batch = 0
nb_houses = []
inputfiles = ['database/15min_houses_pv_042014_052015_v0001.csv']
outputfile = 'result.pickle'

# Simulation period
start= '2014-05-01 00:00:00'
start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
end = '2014-05-10 23:45:00'
end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
f_horizon = timedelta(days=2) - timedelta(minutes=15)
training_period = timedelta(days=14)
one_day = timedelta(hours=23, minutes=45)

# Simulation variable (forecasting method)
f_method = forecast.ma
f_kwargs = {}
control_func = optimization.pool

# Running simulation

In [ ]:
_start_time = datetime.now()
metrics = pandas.DataFrame()

for inputfile, nb_house in zip(inputfiles, nb_houses):
    print(f'Using {inputfile} out of {inputfiles}')
    df = pandas.read_pickle(inputfile)
    df = df.loc[:, ['vo_houses_kW', 'vo_pv_coef']]

    # Assert timing
    assert start in df.index
    assert end in df.index
    assert int(pandas.infer_freq(df.index)[:-1]) == 15

    # Assert overall consumption (20000kWh is a lot)
    assert df['vo_houses_kW'].sum() * 15/60 / nb_houses < 20000, f'Total conso {df["vo_houses_kW"].sum()}'

    # Assert PV
    assert df['vo_pv_coef'].min() >= 0
    assert df['vo_pv_coef'].max() <= 1.01, f'Max PV coef = {df["vo_pv_coef"].max()}'
    
    ###########
    ### RUN ###
    ###########
    pv_capacity = 3 * nb_house  # kW
    battery_kWh = 6 * nb_house   # kWh
    battery_kW = 2 * nb_house  # kW
    result = main(df, start, end, pv_capacity, battery_kWh, battery_kW,
                  f_method, f_kwarg, f_horizon, f_train_period, control_func)
    
    # Save
    metric['index'] = nb_house
    metric['filename'] = inputfile
    metric['algo'] = f'{f_method.__name__}'
    metrics = metrics.append(result, ignore_index=True)
    metric['batch'] = batch

print(tabulate(
    metrics[['index', 'MAPE_%', 'MAPE_9a8p_%', 'MASE',
             'scons_%', 'ssuff_%',
             'ME_kW', 'MAE_kW']],
    headers='keys', tablefmt='psql'))
print('')
print('------')
_time_elapsed = datetime.now() - _start_time
print('Total time elapsed (hh:mm:ss) {}'.format(_time_elapsed))
print('')

# Visualization

In [ ]:
# plot metrics as a function of nb houses

In [ ]:
# Draw relation between value and quality